In [25]:
# %pip install datasets
# %pip install Pillow
# %pip install numpy
# %pip install matplotlib
# %pip install import_ipynb
# %pip install tensorflow
# %pip install tensorflow.keras
# %pip install easyocr
# %pip install pandas

# # library from etl
# %pip install datasets
# %pip install numpy
# %pip install Pillow
# %pip install opencv-python
# %pip install json

In [26]:
import easyocr
import matplotlib.pyplot as plt
from datasets import load_dataset
import numpy as np
from PIL import Image
import cv2
import tensorflow as tf
# from tensorflow.keras import layers
import json
import pandas as pd

In [27]:
from kaggle_secrets import UserSecretsClient
import os

# Ambil GitHub token dari Kaggle Secrets
user_secrets = UserSecretsClient()
myGithub = user_secrets.get_secret("githubKey")

# Clone repo GitHub
!git clone https://{myGithub}@github.com/fitristachan/invoice-image-processing.git invoice_image_processing
!git -C invoice_image_processing pull

fatal: destination path 'invoice_image_processing' already exists and is not an empty directory.
Already up to date.


In [28]:
# from google.colab import userdata

# # Retrieve the GitHub token from userdata
# myGithub = userdata.get('githubKey')

# !git clone https://{myGithub}@github.com/fitristachan/invoice-image-processing.git /content/invoice_image_processing
# !git -C invoice_image_processing pull



In [29]:
# import sys
# import os

# sys.path.append('/content/invoice-image-processing')
# print(os.listdir('/content/invoice-image-processing'))

# !jupyter nbconvert --to script '/content/invoice-image-processing/data_etl_invoice.ipynb' --output data_etl

Image Classification

In [30]:
from invoice_image_processing.data_etl import DatasetReceipt

In [31]:
from keras.applications import ResNet50V2

# keras.applications.ResNet50V2(
#     include_top=True,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
#     name="resnet50v2",
# )

pre_trained_model = ResNet50V2(input_shape = (600, 600, 3),
                                include_top = False,
                                pooling='max'
                                )

pre_trained_model.summary()

Model: "resnet50v2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 600, 600, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 606, 606, 3)    │              0 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 300, 300, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 302, 302, 64)   │              0 │ conv1_conv[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 150, 150, 64)   │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_preact_bn    │ (None, 150, 150, 64)   │            256 │ pool1_pool[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_preact_relu  │ (None, 150, 150, 64)   │              0 │ conv2_block1_preact_b… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 150, 150, 64)   │          4,096 │ conv2_block1_preact_r… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 150, 150, 64)   │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 150, 150, 64)   │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_pad        │ (None, 152, 152, 64)   │              0 │ conv2_block1_1_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 150, 150, 64)   │         36,864 │ conv2_block1_2_pad[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 150, 150, 64)   │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 150, 150, 64)   │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv  

 Total params: 23,564,800 (89.89 MB)

 Trainable params: 23,519,360 (89.72 MB)

 Non-trainable params: 45,440 (177.50 KB)

In [32]:
from datasets import concatenate_datasets
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from PIL import Image
import torch

# Load individual datasets
train_cord = DatasetReceipt("naver-clova-ix/cord-v2", "train")
val_cord = DatasetReceipt("naver-clova-ix/cord-v2", "validation")
train_donut = DatasetReceipt("katanaml-org/invoices-donut-data-v1", "train")
val_donut = DatasetReceipt("katanaml-org/invoices-donut-data-v1", "validation")

class CombinedDataset:
    def __init__(self, dataset, image_size=(600, 600), is_train=True):
        self.dataset = dataset
        self.image_size = image_size
        
        self.transform = A.Compose([
            A.Resize(height=image_size[0], width=image_size[1])
        ])
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        sample = self.dataset[idx]
        
        img = sample.get("image", sample.get("img"))
        if isinstance(img, Image.Image):
            img = np.array(img.convert("RGB"))

        img = self.transform(image=img)["image"]  # [H, W, C], ready buat TF

        return {
            "image": img.astype(np.uint8),
            "quantities": torch.tensor(sample.get("quantities", [])),
            "prices": torch.tensor(sample.get("prices", [])),
            "total_price": torch.tensor(float(sample.get("total_price", 0))),
            "item_names": sample.get("item_names", [])
        }

# Cara penggunaan:
combined_train = concatenate_datasets([train_cord.dataset, train_donut.dataset])
combined_val = concatenate_datasets([val_cord.dataset, val_donut.dataset])

train_dataset = CombinedDataset(combined_train, is_train=True)
val_dataset = CombinedDataset(combined_val, is_train=False)

# Verifikasi
sample = train_dataset[0]
print(f"Image shape: {sample['image'].shape}")  # Harusnya [3, 600, 600]
print(f"Image type: {type(sample['image'])}")  # torch.Tensor

print(f"Total data train CORD: {len(train_cord)}")
print(f"Total data train Donut: {len(train_donut)}")
print(f"Total data gabungan: {len(train_dataset)}")

# Verifikasi jumlah harus sama dengan penjumlahan
assert len(train_dataset) == len(train_cord) + len(train_donut), "Jumlah data tidak sesuai!"

# Ambil satu sampel
sample = train_dataset[0]
plt.imshow(sample["image"])
plt.axis("off")
plt.title("Albumentation Resize Only")
plt.show()

FileNotFoundError: Couldn't find any data file at /kaggle/working/naver-clova-ix/cord-v2. Couldn't find 'naver-clova-ix/cord-v2' on the Hugging Face Hub either: LocalEntryNotFoundError: An error happened while trying to locate the file on the Hub and we cannot find the requested files in the local cache. Please check your connection and try again or make sure your Internet connection is on.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd

def dataset_generator(dataset, batch_size=16, max_items=10, max_name_length=20, vocab_size=128):
    while True:
        batch_images = []
        batch_names = []
        batch_quantities = []
        batch_prices = []
        batch_totals = []

        for _ in range(batch_size):
            sample = dataset[np.random.randint(len(dataset))]
            img = sample["image"]

            # Augmentasi aman untuk dokumen
            img = tf.convert_to_tensor(img, dtype=tf.float32)
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_brightness(img, max_delta=0.1)
            img = tf.image.random_contrast(img, lower=0.9, upper=1.1)
            img = tf.image.resize(img, (600, 600))  # Resize agar konsisten
            img = img.numpy()

            batch_images.append(img)

            # Inisialisasi padding
            padded_names = np.zeros((max_items, max_name_length), dtype=np.int32)
            padded_quantities = np.zeros((max_items, max_name_length), dtype=np.int32)
            padded_prices = np.zeros((max_items, max_name_length), dtype=np.int32)
            padded_total = np.zeros(max_name_length, dtype=np.int32)

            if "item_names" in sample and len(sample["item_names"]) > 0:
                item_names = sample["item_names"]
                quantities = sample.get("quantities", ["0"] * len(item_names))
                prices = sample.get("prices", ["0"] * len(item_names))
                total_price = sample.get("total_price", "0")

                # Cast to string dulu
                quantities = [str(q) for q in quantities]
                prices = [str(p) for p in prices]
                total_price = str(total_price)

                n_items = min(len(item_names), max_items)

                for i in range(n_items):
                    name = str(item_names[i])[:max_name_length]
                    quantity = quantities[i][:max_name_length]
                    price = prices[i][:max_name_length]

                    name_token = [ord(c) % vocab_size for c in name]
                    qty_token = [ord(c) % vocab_size for c in quantity]
                    price_token = [ord(c) % vocab_size for c in price]

                    padded_names[i, :len(name_token)] = name_token
                    padded_quantities[i, :len(qty_token)] = qty_token
                    padded_prices[i, :len(price_token)] = price_token

                # Total price
                total_token = [ord(c) % vocab_size for c in total_price[:max_name_length]]
                padded_total[:len(total_token)] = total_token

            batch_names.append(padded_names)
            batch_quantities.append(padded_quantities)
            batch_prices.append(padded_prices)
            batch_totals.append(padded_total)

        # Convert ke numpy
        batch_images = np.array(batch_images, dtype=np.float32)
        batch_names = to_categorical(batch_names, num_classes=vocab_size)
        batch_quantities = np.array(batch_quantities, dtype=np.float32)[:, :, 0]
        batch_prices = np.array(batch_prices, dtype=np.float32)[:, :, 0]
        batch_totals = np.array(batch_totals, dtype=np.float32)[:, 0]

        yield (batch_images, {
            "item_name": batch_names,
            "quantity": batch_quantities,
            "price": batch_prices,
            "total_price": batch_totals
        })


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.applications import ResNet50V2
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.utils import to_categorical
# import numpy as np
# import pandas as pd

# def dataset_generator(dataset, batch_size=16, max_items=10, max_name_length=20, vocab_size=128):
#     while True:
#         batch_images = []
#         batch_names = []
#         batch_quantities = []
#         batch_prices = []
#         batch_totals = []

#         for _ in range(batch_size):
#             sample = dataset[np.random.randint(len(dataset))]
            
#             img = sample["image"]
#             if img.ndim == 4 and img.shape[0] == 1:
#                 img = img[0]
#             batch_images.append(img)

#             padded_names = np.zeros((max_items, max_name_length), dtype=np.int32)
#             padded_quantities = np.zeros((max_items, max_name_length), dtype=np.int32)
#             padded_prices = np.zeros((max_items, max_name_length), dtype=np.int32)
#             padded_total = np.zeros(max_name_length, dtype=np.int32)

#             if "item_names" in sample and len(sample["item_names"]) > 0:
#                 item_names = sample["item_names"]
#                 quantities = sample.get("quantities", ["0"] * len(item_names))
#                 prices = sample.get("prices", ["0"] * len(item_names))
#                 total_price = sample.get("total_price", "0")

#                 # Cast to string first
#                 quantities = [str(q) for q in quantities]
#                 prices = [str(p) for p in prices]
#                 total_price = str(total_price)

#                 n_items = min(len(item_names), max_items)

#                 for i in range(n_items):
#                     name = str(item_names[i])[:max_name_length]
#                     quantity = quantities[i][:max_name_length]
#                     price = prices[i][:max_name_length]

#                     name_token = [ord(c) % vocab_size for c in name]
#                     qty_token = [ord(c) % vocab_size for c in quantity]
#                     price_token = [ord(c) % vocab_size for c in price]

#                     padded_names[i, :len(name_token)] = name_token
#                     padded_quantities[i, :len(qty_token)] = qty_token
#                     padded_prices[i, :len(price_token)] = price_token

#                 # Total price
#                 total_token = [ord(c) % vocab_size for c in total_price[:max_name_length]]
#                 padded_total[:len(total_token)] = total_token

#             batch_names.append(padded_names)
#             batch_quantities.append(padded_quantities)
#             batch_prices.append(padded_prices)
#             batch_totals.append(padded_total)

#         # Convert to numpy arrays
#         batch_images = np.array(batch_images, dtype=np.float32)
        
#         # Only item_name is categorical (sequence of one-hot)
#         batch_names = to_categorical(batch_names, num_classes=vocab_size)
        
#         # The others are numeric regression targets
#         batch_quantities = np.array(batch_quantities, dtype=np.float32)[:, :, 0]
#         batch_prices = np.array(batch_prices, dtype=np.float32)[:, :, 0]
#         batch_totals = np.array(batch_totals, dtype=np.float32)[:, 0]


#         # # Optional: one-hot encode
#         # batch_names = to_categorical(batch_names, num_classes=vocab_size)
#         # batch_quantities = to_categorical(batch_quantities, num_classes=vocab_size)
#         # batch_prices = to_categorical(batch_prices, num_classes=vocab_size)
#         # batch_totals = to_categorical(batch_totals, num_classes=vocab_size)

#         yield (batch_images, {
#             "item_name": batch_names,
#             "quantity": batch_quantities,
#             "price": batch_prices,
#             "total_price": batch_totals
#         })

 

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda, LSTM, Dense, TimeDistributed, RepeatVector
from tensorflow.keras import backend as K
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout, Input
import tensorflow as tf

# Training hyperparameters
INITIAL_LR = 1e-5
SECOND_LR = 1e-6
MOMENTUM = 0.9
GRADIENT_CLIP_NORM = 1.0
STD_DEV = 0.01
EPOCHS = 25
STEPS_PER_EPOCH = 1000
NMS_IOU_THRESHOLD = 0.8
NUM_CHAR_CLASSES = 122

# 1. Define Input Layer
inputs = Input(shape=(600, 600, 3))

# 2. Load Pretrained ResNet50V2
base_model = ResNet50V2(
    input_shape=(600, 600, 3),
    include_top=False,
    pooling='max'  # GlobalMaxPooling2D
)
base_model.trainable = False

# 3. Connect Base Model to Inputs
x = base_model(inputs)

# 4. Add Custom Layers with Proper Initialization
kernel_initializer = tf.keras.initializers.RandomNormal(stddev=STD_DEV)
x = Dense(512, activation='relu', kernel_initializer=kernel_initializer)(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', 
          kernel_regularizer=regularizers.l2(0.01),
          kernel_initializer=kernel_initializer)(x)

# For price/quantity outputs, add reasonable constraints:
price_output = Dense(10, activation='relu', name='price')(x)  # Prevents negative prices
quantity_output = Dense(10, activation='relu', name='quantity')(x)     # Prevents negative quantities

# Total amount regression
total_output = Dense(1, name="total_price", kernel_initializer=kernel_initializer)(x)

sequence_input = RepeatVector(10)(Dense(256, activation='relu')(x))  # (None, 10, 256)
sequence_output = TimeDistributed(RepeatVector(20))(sequence_input)  # (None, 10, 20, 256)

# LSTM per item + karakter
lstm_output = TimeDistributed(LSTM(128, return_sequences=True))(sequence_output)  # (None, 10, 20, 128)

# Output klasifikasi per karakter (pakai softmax)
name_output = TimeDistributed(TimeDistributed(Dense(NUM_CHAR_CLASSES, activation='softmax')), name='item_name')(lstm_output)  # (None, 10, 20, 122)

# Create model
model = Model(
    inputs=inputs,
    outputs=[
        name_output,  
        quantity_output,
        price_output,
        total_output
    ]
)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=INITIAL_LR, 
    beta_1=MOMENTUM,   
    beta_2=0.999, 
    epsilon=1e-7, 
    clipnorm=GRADIENT_CLIP_NORM
)

In [ ]:
loss_fn = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
model.compile(
    optimizer=optimizer,
    loss={
        "item_name": loss_fn,
        "quantity": "mse",
        "price": "mse",
        "total_price": "mse",
    },
    metrics={
        "item_name": "accuracy",
        "quantity": "mae",
        "price": "mae",
        "total_price": "mae",
    },
    loss_weights={
        "item_name": 1.5,
        "quantity": 0.8,
        "price": 1.0,
        "total_price": 1.2,
    }
)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import (ModelCheckpoint, LearningRateScheduler, 
                                      TensorBoard, CSVLogger, Callback)
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau

# 1. Enhanced Model Checkpoint
checkpoint_path = "/content/drive/MyDrive/dataset/model/model_checkpoint_resnet50v2.keras"
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,  # Now saving only best model
    monitor='val_loss',  # Monitoring validation loss
    mode='min',
    save_weights_only=False,
    save_freq='epoch',
    verbose=1
)

def lr_schedule(epoch):
    if epoch < 10:
        return 1e-5
    elif epoch < 20:
        return 1e-6
    else:
        return 1e-8

lr_callback = LearningRateScheduler(lr_schedule, verbose=1)

class EnhancedTrainingCallback(Callback):
    def __init__(self, patience=5, min_epoch_for_acc_stop=5):
        super(EnhancedTrainingCallback, self).__init__()
        self.patience = patience
        self.min_epoch_for_acc_stop = min_epoch_for_acc_stop
        self.wait = 0
        self.stopped_epoch = 0
        self.best_loss = float('inf')
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}

        current_acc = logs.get('item_name_accuracy', 0)
        current_val_acc = logs.get('val_item_name_accuracy', 0)

        # Condition 1: Accuracy >= 95% AND epoch >= min_epoch_for_acc_stop
        if (current_acc >= 0.95 and current_val_acc >= 0.95 and epoch + 1 >= self.min_epoch_for_acc_stop):
            print(f"\nAccuracy reached 95%% and epoch ≥ {self.min_epoch_for_acc_stop}. Stopping training.")
            self.model.stop_training = True
            self.stopped_epoch = epoch
            return
        
        # Condition 2: Loss plateau detection
        current_loss = logs.get('loss', float('inf'))
        if current_loss < self.best_loss:
            self.best_loss = current_loss
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                print(f"\nTraining stopped early after {self.patience} epochs without improvement.")
                self.model.stop_training = True
                self.stopped_epoch = epoch

stop_callback = EnhancedTrainingCallback(patience=5)

lr_plateau = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-5
)

# 4. CSV Logging (corrected version)
csv_logger = CSVLogger(
    'training_log.csv',
    separator=',',
    append=False  # Change to True if you want to continue previous logs
)

# 5. TensorBoard Integration
tensorboard_callback = TensorBoard(
    log_dir='./logs',
    histogram_freq=1,
    write_graph=True,
    write_images=False,
    update_freq='epoch',
    profile_batch=0  # Disable profiling for cleaner logs
)

# Combine all callbacks
final_callbacks = [
    checkpoint_callback,
    lr_callback,
    stop_callback,
    csv_logger,
    tensorboard_callback
]

final_callbacks.append(lr_plateau)

In [ ]:
# Data Augmentation
train_generator = dataset_generator(train_dataset, vocab_size=122)
val_generator = dataset_generator(val_dataset, vocab_size=122)

In [ ]:
print(model.output_names)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=500,  # Adjust based on validation set size
    callbacks=final_callbacks,
    verbose=1
)

In [ ]:
# from tensorflow.keras.models import load_model

# model = load_model("/content/drive/MyDrive/dataset/model/model_checkpoint_resnet50v2.keras", compile=True)

In [ ]:
# history = model.fit(
#     train_generator,
#     steps_per_epoch=STEPS_PER_EPOCH,
#     initial_epoch=15,  # Karena sebelumnya udah sampai epoch 15
#     epochs=EPOCHS,
#     validation_data=val_generator,
#     validation_steps=500,  # Adjust based on validation set size
#     callbacks=final_callbacks,
#     verbose=1
# )

In [ ]:
# history = model.fit(
#     train_generator,
#     steps_per_epoch=STEPS_PER_EPOCH,
#     initial_epoch=19, 
#     epochs=EPOCHS,
#     validation_data=val_generator,
#     validation_steps=500,  # Adjust based on validation set size
#     callbacks=final_callbacks,
#     verbose=1
# )

In [ ]:
with open("training_log.csv") as f:
    for i, line in enumerate(f):
        if i == 0: 
            print("Header:", len(line.split(',')), "columns")
        if i == 26:  # Baris 27 (indeks 0-based)
            print("Line 27:", len(line.split(',')), "columns")
            print("Content:", line)
            break

In [ ]:
from io import StringIO
import pandas as pd
import matplotlib.pyplot as plt

def clean_log_file(path):
    with open(path) as f:
        # Ambil header (baris pertama)
        header = f.readline().strip()
        # Filter baris yang memiliki 12 koma (13 kolom)
        lines = [line.strip() for line in f if line.count(',') == 12]
        
    # Gabungkan header dengan data yang sudah difilter
    cleaned_data = [header] + lines
    return pd.read_csv(StringIO('\n'.join(cleaned_data)))

# Baca dan bersihkan data
try:
    log_data = clean_log_file("training_log.csv")
except Exception as e:
    print(f"Error: {e}")
    # Fallback: baca dengan engine python
    log_data = pd.read_csv("training_log.csv", engine='python', on_bad_lines='warn')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load training log
log_data = pd.read_csv("training_log.csv")

plt.figure(figsize=(15, 10))

# Plot Item Name Accuracy
plt.subplot(2, 2, 1)
plt.plot(log_data['epoch'], log_data['item_name_accuracy'], 'b-', label='Name Training')
plt.plot(log_data['epoch'], log_data['val_item_name_accuracy'], 'r-', label='Name Validation')
plt.title('Item Name Accuracy')
plt.legend()

# Plot Total Loss (di subplot berbeda)
plt.subplot(2, 2, 2)
plt.plot(log_data['epoch'], log_data['loss'], 'b-', label='Training Loss')
plt.plot(log_data['epoch'], log_data['val_loss'], 'r-', label='Validation Loss')
plt.title('Total Loss')
plt.legend()

# Plot Regression Performance (Price MAE)
plt.subplot(2, 2, 3)
plt.plot(log_data['epoch'], log_data['price_mae'], 'g-', label='Train Price MAE')
plt.plot(log_data['epoch'], log_data['val_price_mae'], 'm-', label='Val Price MAE')
plt.title('Price MAE')
plt.legend()

# Plot Component Losses
plt.subplot(2, 2, 4)
plt.plot(log_data['epoch'], log_data['item_name_loss'], 'c-', label='Item Name Loss')
plt.plot(log_data['epoch'], log_data['price_loss'], 'y-', label='Price Loss')
plt.plot(log_data['epoch'], log_data['quantity_loss'], 'orange', label='Quantity Loss')
plt.plot(log_data['epoch'], log_data['total_price_loss'], 'purple', label='Total Price Loss')
plt.title('Component Losses')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import os

In [ ]:
# Save Model
model.save("faster_rcnn_resnet50v2.h5")
model.save("faster_rcnn_resnet50v2.keras")
print("Model saved successfully!")

In [ ]:
drive_service = build('drive', 'v3')

folder_id = "1a0i_DpxSo1RnkT7ZjUr_AdawVvW2vBSX"  # Dapatkan dari URL Google Drive folder kamu

file_metadata = {
    "name": "faster_rcnn_resnet50v2.keras",
    "parents": [folder_id]
}
media = MediaFileUpload("faster_rcnn_resnet50v2.keras", mimetype="application/octet-stream")

file = drive_service.files().create(body=file_metadata, media_body=media, fields="id").execute()
print(f"File uploaded with ID: {file.get('id')}")

file_metadata_h5 = {
    "name": "faster_rcnn_resnet50v2.h5",
    "parents": [folder_id]
}
media_h5 = MediaFileUpload("faster_rcnn_resnet50v2.h5", mimetype="application/octet-stream")

file_h5 = drive_service.files().create(body=file_metadata_h5, media_body=media_h5, fields="id").execute()
print(f"File uploaded with ID: {file_h5.get('id')}")
